In [108]:
import yfinance as yf
import pandas as pd
import numpy as np
import statsmodels.api as sm
import requests as re
import plotly.graph_objects as go
from datetime import datetime
import matplotlib.pyplot as plt


In [109]:
df = pd.read_csv('ind_nifty50list.csv')
tickers = (df['Symbol'] + ['.NS']).tolist() + ['^NSEI']

In [110]:
try:
    data = yf.download(
        tickers,
        period = "5y",
        auto_adjust = True,
        group_by= 'ticker',
        threads = True,
        progress= False
    )
    print('DOWNLOAD COMPLETE')
except Exception as e:
    print("ERROR FETCHING : {e}")


1 Failed download:
['DUMMYHDLVR.NS']: YFPricesMissingError('possibly delisted; no price data found  (period=5y) (Yahoo error = "No data found, symbol may be delisted")')


DOWNLOAD COMPLETE


In [111]:
if len(tickers) >1:
    closing_price = pd.DataFrame()
    for ticker in tickers:
        try:
            if ticker in data.columns.get_level_values(0):
                closing_price[ticker] = data[ticker]['Close']
        except:
            continue

else:
    closing_price = data[['Close']].copy()
    closing_price.columns = tickers

criteria = 100
valid_tickers = closing_price.columns[closing_price.isna().sum() < criteria ]
closing_price = closing_price[valid_tickers]

print(f'only {len(valid_tickers)} tickers had sufficient data')

only 48 tickers had sufficient data


In [112]:
closing_price

,ADANIENT.NS,ADANIPORTS.NS,APOLLOHOSP.NS,ASIANPAINT.NS,AXISBANK.NS,BAJAJ-AUTO.NS,BAJFINANCE.NS,BAJAJFINSV.NS,BEL.NS,BHARTIARTL.NS,...,SUNPHARMA.NS,TCS.NS,TATACONSUM.NS,TATASTEEL.NS,TECHM.NS,TITAN.NS,TRENT.NS,ULTRACEMCO.NS,WIPRO.NS,^NSEI
Date,,,,,,,,,,,,,,,,,,,,,
2021-01-18,506.619385,500.404816,2578.993652,2477.350830,654.268188,3116.432129,462.339172,833.584656,40.982525,563.089417,...,552.192200,2832.166260,562.554749,57.394703,837.876221,1480.522583,656.551880,5240.513184,204.042465,14281.299805
2021-01-19,528.815491,517.198120,2614.823242,2531.765625,665.969299,3183.985107,485.780182,890.411743,42.056568,568.221863,...,566.532227,2866.406250,560.582642,58.634369,833.050110,1496.622070,674.158081,5341.801270,203.427795,14521.150391
2021-01-20,545.425110,542.242737,2644.772217,2581.729980,672.940247,3186.564941,488.013580,891.818604,41.580917,571.915405,...,564.775452,2908.689941,574.003113,59.413460,855.670410,1507.847534,676.346436,5397.261719,210.378159,14644.700195
2021-01-21,535.598999,531.613403,2555.073975,2600.011963,673.039795,3238.465820,501.369904,903.247925,41.089928,557.717041,...,553.569153,2877.966309,572.656250,57.394703,843.290100,1494.061890,663.067139,5393.710449,210.780045,14590.349609
2021-01-22,525.523438,520.110352,2567.774902,2485.438721,641.820068,3576.190186,487.092743,905.063843,40.614277,553.400024,...,546.256653,2903.678955,562.554749,55.698769,823.271729,1468.361572,656.999512,5385.293457,210.756363,14371.900391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026-01-12,2171.600098,1443.500000,7268.500000,2896.399902,1274.199951,9491.000000,951.900024,1998.900024,417.700012,2044.000000,...,1736.000000,3181.759277,1192.300049,183.240005,1586.099976,4231.600098,4056.399902,12098.000000,263.100006,25790.250000
2026-01-13,2158.500000,1428.599976,7311.500000,2886.300049,1262.000000,9554.000000,949.000000,2011.199951,413.700012,2026.900024,...,1728.699951,3209.652100,1189.400024,182.570007,1614.800049,4239.200195,3921.899902,12044.000000,264.200012,25732.300781
2026-01-14,2153.300049,1430.000000,7272.500000,2813.899902,1298.800049,9579.500000,945.950012,2000.900024,417.600006,2022.500000,...,1700.699951,3135.500000,1171.400024,189.250000,1588.500000,4221.500000,3932.199951,12255.000000,260.200012,25665.599609


In [113]:
returns = np.log(closing_price/closing_price.shift(1)).dropna()
market_returns = returns['^NSEI']
stock_returns = returns.drop('^NSEI', axis = 1)

print(f"calculated market returns and stock returns for {len(stock_returns.columns)} stocks")
print(f'Time period:  {returns.index[0].strftime('%Y-%m')} to {returns.index[-1].strftime('%Y-%m')} ')

calculated market returns and stock returns for 47 stocks
Time period:  2021-01 to 2026-01 


In [114]:
annual_rfr = 0.05624
daily_rf = (1+annual_rfr)**(1/365) - 1
daily_rf_log = np.log(1 + daily_rf)   #force of interest

excess_market = market_returns - daily_rf_log
excess_stocks = stock_returns - daily_rf_log

print(f'daily RFR: {daily_rf_log:.10f} ({daily_rf*100:.5f}%) ')

daily RFR: 0.0001499053 (0.01499%) 


In [115]:
results_list = []
for ticker in excess_stocks.columns:
    try :
        y = excess_stocks[ticker].dropna()
        x = excess_market.loc[y.index]

        if len(y) < 12: ##for the sake of sufficiency, tho we already did this above
            continue

        x_with_const = sm.add_constant(x)

        # Running OLS

        model = sm.OLS(y,x_with_const).fit()

        alpha = model.params['const']
        beta = model.params['^NSEI']
        alpha_pval = model.pvalues['const']
        r_sq = model.rsquared

        results_list.append({
            'Ticker' : ticker.replace('-Close', ''),
            'Alpha' : alpha,
            'Annual Alpha': alpha*365,
            'Annual Alpha %': np.exp(alpha*365) - 1,
            'Alpha P value': alpha_pval,
            'Beta': beta,
            'R_squared': r_sq,
            'Observations': len(y)
        })

    except Exception as e:
        continue
results_df = pd.DataFrame(results_list)
results_df = results_df.sort_values('Alpha', ascending = False).reset_index(drop=True) 

print('Analysis complete')

Analysis complete


In [116]:
top_20 = results_df.head(20).copy()
bottom_20 = results_df.tail(20).copy()
plot_data = pd.concat([bottom_20, top_20])

plot_data['Alpha_Annual_pct'] = plot_data['Annual Alpha']*100

colors = ['red' if x<0 else 'blue' for x in plot_data['Alpha_Annual_pct']]

fig = go.Figure()

fig.add_trace(go.Bar(
    x = plot_data['Alpha_Annual_pct'],
    y =plot_data['Ticker'], 
    orientation = 'h',
    marker  = dict(
        color = colors,
        line = dict(color = 'black', width = 0.5)
    ),
    text = plot_data['Alpha_Annual_pct'].round(2),
    textposition = 'outside',
    hovertemplate='<b>%{y}</b><br>' +
                  'Annual Alpha: %{x:.2f}%<br>' +
                  'Beta: %{customdata[0]:.2f}<br>' +
                  'R²: %{customdata[1]:.3f}<br>' +
                  '<extra></extra>',
    customdata=plot_data[['Beta', 'R_squared']].values
))
fig.update_layout(
    title={
        'text': 'S&P 500 CAPM Analysis: Top & Bottom 20 Alphas',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 24, 'color': '#1f1f1f'}
    },
    xaxis_title='Annualized Alpha (%)',
    yaxis_title='',
    height=900,
    width=1200,
    template='plotly_white',
    xaxis=dict(
        showgrid=True,
        gridcolor='lightgray',
        zeroline=True,
        zerolinewidth=2,
        zerolinecolor='black'
    ),
    yaxis=dict(
        tickfont=dict(size=11)
    ),
    hovermode='y',
    margin=dict(l=100, r=100, t=100, b=80)
)

fig.show()

In [117]:
print("\n" + "=" * 60)
print("SUMMARY STATISTICS")
print("=" * 60)

print("\nTop 5 Alphas (Annualized %):")
print(results_df[['Ticker', 'Annual Alpha', 'Beta', 'R_squared']].head().to_string(index=False))

print("\nBottom 5 Alphas (Annualized %):")
print(results_df[['Ticker', 'Annual Alpha', 'Beta', 'R_squared']].tail().to_string(index=False))

print("\nOverall Statistics:")
print(f"  Mean Alpha (Annual):   {results_df['Annual Alpha'].mean()*100:.2f}%")
print(f"  Median Alpha (Annual): {results_df['Annual Alpha'].median()*100:.2f}%")
print(f"  Mean Beta:             {results_df['Beta'].mean():.3f}")
print(f"  Mean R²:               {results_df['R_squared'].mean():.3f}")

# Count significant alphas
sig_positive = (results_df['Alpha'] > 0) & (results_df['Alpha P value'] < 0.05)
sig_negative = (results_df['Alpha'] < 0) & (results_df['Alpha P value'] < 0.05)

print(f"\n  Significant positive alphas (p<0.05): {sig_positive.sum()}")
print(f"  Significant negative alphas (p<0.05): {sig_negative.sum()}")

print("\n" + "=" * 60)
print("Analysis complete!")
print("=" * 60)


SUMMARY STATISTICS

Top 5 Alphas (Annualized %):
      Ticker  Annual Alpha     Beta  R_squared
      BEL.NS      0.502550 1.124585   0.229114
MAXHEALTH.NS      0.428330 0.565032   0.048408
    TRENT.NS      0.344753 1.073011   0.188458
COALINDIA.NS      0.292562 1.010522   0.231152
     NTPC.NS      0.270201 0.955637   0.264613

Bottom 5 Alphas (Annualized %):
       Ticker  Annual Alpha     Beta  R_squared
     WIPRO.NS     -0.105476 1.008192   0.301802
ASIANPAINT.NS     -0.106906 0.665318   0.169029
       TCS.NS     -0.117407 0.778583   0.278104
  HDFCLIFE.NS     -0.124028 0.820833   0.219394
 KOTAKBANK.NS     -0.131678 0.980724   0.346966

Overall Statistics:
  Mean Alpha (Annual):   10.61%
  Median Alpha (Annual): 12.01%
  Mean Beta:             0.960
  Mean R²:               0.267

  Significant positive alphas (p<0.05): 1
  Significant negative alphas (p<0.05): 0

Analysis complete!
